<a href="https://colab.research.google.com/github/otabekds/Web-scrapping-project/blob/main/islom_uz_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
# This is my first web scrap project. In doing so, I filter out the latest article from the News section of the Islam.uz site and publish the most used words in graphical form.import requests
from bs4 import BeautifulSoup
import itertools
import pandas as pd
import plotly.express as px

In [87]:
def respons_get(url):
    return requests.get(url)

In [88]:
def extract(respons):
    soup = BeautifulSoup(respons,'html.parser')
    soup2 = soup.find_all("div", class_="news_title")
    index = soup.find_all("div", class_="info_c")
    url_respons, name_respons, ind = [], [], 0
    for n in index:
        w = n.text
        if w == "Мақола":
            break
        ind += 1

    for i in soup2:
        url_respons.append(i.find('a')['href'])
        name_respons.append(i.text)

    return name_respons[ind], url_respons[ind]  # index

In [89]:
def extract2(respons2):
    soup = BeautifulSoup(respons2.text.strip(), "html.parser")
    return soup

In [90]:
def transform(soup):
    result_article = []

    article = soup.find_all('p')
    res = [x.text.split('.') for x in article]  # I split it .ta bn
    res = res[1:]   #I removed the sentence at the beginning of the article because it's all at the beginning of the article
    res = [[i.split()[:-1] for i in x] for x in res]  # In this case, I omitted the verbs that come at the end of the sentence
    res = list(itertools.chain.from_iterable(res))
    stop_words = ['унинг','жуда','бири','билан','агар','топа','сени','олиб','учун','бирга','бир','осон','олти','мени','энди','азиз','мана','менга','асоси'] 
    kic = {-4 :'нинг', -2 :'га', -2 :'ди', -2 : 'да', -4 : 'нинг', -2 : "ни" , -3 : "дан", -3 : "ган", -2 : "ни"}
    for i in res:   # In this loop we can clear the words
      for x in i:
        if len(x) <= 3:    
          pass
        elif x.lower() in stop_words:
          pass
        elif x.isalpha():
          result_article.append(x.lower())

    for d in range(5):
      for i in result_article[:]:  #in this loop we remove the suffixes of the words
        for k , v in kic.items():
          if i[k:] == v:
                g = result_article.index(i)
                h = result_article.pop(g)
                result_article.append(h[:k])
              
    return result_article

In [91]:
def count_words(clean_data):
  res,res2 = [],[]
  for x in clean_data:
    w = clean_data.count(x)
    if x not in res:
      res.append(x)
      res2.append(w)
  return res, res2

In [92]:

def top10(ready_words):
  df = pd.DataFrame(data=ready_words[0], index=ready_words[1])
  df = df.sort_index()
  df.columns = ['Maqoladagi sozlar']
  df.index.name = "Sozlarning soni"
  data = df.tail(10).reset_index()
  data.index.name = 'Miqdori'
  return data

In [93]:
def main():
    respons =  respons_get("https://islom.uz").text
    url1 = extract(respons)
    a_name = url1[0]
    respons2 = respons_get(f"https://islom.uz{url1[1]}")
    soup = extract2(respons2)
    clean_data = transform(soup)
    ready_words = count_words(clean_data)
    pd_words = top10(ready_words)

    fig = px.bar(pd_words, x='Maqoladagi sozlar', y="Sozlarning soni", color=pd_words.index, title=f"Мақола номи:   {a_name}", width=1200, height=500)
    fig.show()
main()